<h1> Description </h1>
The Purpose of this Notebook is to use a branch and bound algorithm to select the optimal lineup given a set of baseball players in the form of:
<table>
<tr> 
    <th>POS</th>     
    <th>Player</th>   
    <th>EFP(Expected Fantasy Points)</th>
    <th>Salary</th>
</tr>
<tr>
    <th>String</th>  
    <th>String</th>
    <th>float</th>
    <th>int</th>
</table>
<p>
and the Salary Cap (int)
</p>
<p>

The program will then choose the lineup that will generate the most expected fantasy points
</p>

<h2>Loading the Excel File </h2>
<p>
    For a given dfs league, we must first load the draftkings data from a pre-downloaded excel sheet. Make sure that it is stored in the directory /Users/imnotcreative1/Downloads
</p>
<p>
    To do this we will use the openpxyl library
</p> 
<p> Note: Every downloaded sheet must be named: "DKSalaries.csv" and found in my downloads folder <p>

<p> When grabbing the information from draftkings note the cell structure is as follows: 
<br>
    "A" = Position
<br>
    "B" = Name
<br>
    "C" = Salary
<br>
    "E" = Average Points
</p>
<p> During this process we created another column to store efp/cost </p>

In [96]:
'''
Things that need to be manually entered
'''
#folder directory where the draftkings csv is stored
folder_directory = "/Users/imnotcreative1/Downloads/"

#downloaded file name
dl_file_name = "DKSalaries"

#salary cap
salary_cap = 25000

#number of players, will adjust for positions later
player_limit = 3


In [115]:
'''
    The openpyxl library only supports file with the extension .xlsx or other excel extentions so we need to 
    rename the file to have the .xlsx extension
'''

#function to convert .csv to .xlsx
import os
import glob
import csv
from xlsxwriter.workbook import Workbook

for csvfile in glob.glob(os.path.join(folder_directory, '*.csv')):
    workbook = Workbook(csvfile + '.xlsx')
    worksheet = workbook.add_worksheet()
    with open(csvfile, 'rb') as f:
        reader = csv.reader(f)
        for r, row in enumerate(reader):
            for c, col in enumerate(row):
                worksheet.write(r, c, col)
    workbook.close()

import os

#flag to see if a file was found
dfFile_found = False

#string to hold the filename with an excel extension
dkFile = ""

for filename in os.listdir(folder_directory):
    if filename.startswith(dl_file_name + "."):
        dfFile_found = True
        fileNamePrefix = filename.split('.')[0] #rename file
        os.rename(folder_directory + filename, folder_directory + fileNamePrefix + ".xlsx")
        
if (not dfFile_found):
    print "failed to find draftkings salary file"

#imports the library
import openpyxl

#open the excel data
wb = openpyxl.load_workbook(folder_directory + dl_file_name + '.xlsx')
sheet = wb.get_sheet_by_name('Sheet1')

#load the data into a list of dictionaries
df_data = []

#loop through each row adding each set of data to the list
for row_num in range (2, 10):
    df_dict = {"position": "", "name": "", "salary": 1, "efp": 1, "efp/$1000000": 1}
    df_dict["position"] = sheet["A" + str(row_num)].value.encode('utf-8')
    df_dict["name"] = sheet["B" + str(row_num)].value.encode('utf-8')
    salary =  int(sheet["C" + str(row_num)].value.encode('utf-8'))
    df_dict["salary"] = salary
    efp =  -1 * float(sheet["E" + str(row_num)].value.encode('utf-8'))
    df_dict["efp"] = efp
    df_dict["efp/$1000000"] = int(efp/salary * 1000000)
    print df_dict["efp/$1000000"] 
    df_data.append(df_dict)

print df_data



-2003
-1908
-1990
-1657
-1979
-1856
-2078
-1934
[{'salary': 14400, 'position': 'SP', 'efp': -28.847, 'name': 'Clayton Kershaw', 'efp/$1000000': -2003}, {'salary': 13700, 'position': 'SP', 'efp': -26.145, 'name': 'Max Scherzer', 'efp/$1000000': -1908}, {'salary': 13000, 'position': 'SP', 'efp': -25.872, 'name': 'Jose Fernandez', 'efp/$1000000': -1990}, {'salary': 12300, 'position': 'SP', 'efp': -20.388, 'name': 'Carlos Carrasco', 'efp/$1000000': -1657}, {'salary': 12300, 'position': 'SP', 'efp': -24.35, 'name': 'Johnny Cueto', 'efp/$1000000': -1979}, {'salary': 12200, 'position': 'SP', 'efp': -22.646, 'name': 'Rich Hill', 'efp/$1000000': -1856}, {'salary': 12000, 'position': 'SP', 'efp': -24.941, 'name': 'Stephen Strasburg', 'efp/$1000000': -2078}, {'salary': 11300, 'position': 'SP', 'efp': -21.856, 'name': 'Corey Kluber', 'efp/$1000000': -1934}]


<h2> Negative Expected Fantasy Points??? </h2>
<p> As it turns out python doesn't have a built-in max heap library but does have a built-in min heap library. We would need a max heap if we were looking to maximize efp. In the effort to save time (not coding up our own max heap), we made all the efp negative so instead of looking to maximize efp we are now looking to minimize it. This allows us to use the heapq library. 
</p>



<h2> Greedy Heuristic </h2>
<p> Now that we have all the data in a list of dictionaries, we can now start working on the algorithm. 
<br>
    First we need to apply a greedy heuristic to create a feasible line up which will give us our initial lower bound
    that we will compare future lineups to.
<br>
    We will use the efp/\$1000000 field and choose all the players with the best efp/\$1000000
</p>

In [116]:
#before we start grabing the most valuable players we need to sort the list of players
#this we allow us to grab an initial lower bound

#check accuracy of this section later 

#global lower bound
global_lower_bound = 0

from operator import itemgetter
def sortLODBy(attribute, df_data):
    return sorted(df_data, key=itemgetter(attribute), reverse=False) 

#call sort function
df_data = sortLODBy("efp/$1000000", df_data) 

#remaining funds
funds_remaining = salary_cap

#list_of_players
players_taken = []

#index of sorted list
player_index = 0

while (funds_remaining >= 0):
    players_taken.append(df_data[player_index])
    global_lower_bound +=  df_data[player_index]["efp"]
    print global_lower_bound
    funds_remaining -= df_data[player_index]["salary"]
    player_index+=1

#pop the last player that put it over the salary cap and remove the added efp from the global lower bound
global_lower_bound -= players_taken[len(players_taken)-1]["efp"]
print  df_data
players_taken.pop(len(players_taken)-1)

print players_taken
print global_lower_bound


-24.941
-53.788
[{'salary': 12000, 'position': 'SP', 'efp': -24.941, 'name': 'Stephen Strasburg', 'efp/$1000000': -2078}, {'salary': 14400, 'position': 'SP', 'efp': -28.847, 'name': 'Clayton Kershaw', 'efp/$1000000': -2003}, {'salary': 13000, 'position': 'SP', 'efp': -25.872, 'name': 'Jose Fernandez', 'efp/$1000000': -1990}, {'salary': 12300, 'position': 'SP', 'efp': -24.35, 'name': 'Johnny Cueto', 'efp/$1000000': -1979}, {'salary': 11300, 'position': 'SP', 'efp': -21.856, 'name': 'Corey Kluber', 'efp/$1000000': -1934}, {'salary': 13700, 'position': 'SP', 'efp': -26.145, 'name': 'Max Scherzer', 'efp/$1000000': -1908}, {'salary': 12200, 'position': 'SP', 'efp': -22.646, 'name': 'Rich Hill', 'efp/$1000000': -1856}, {'salary': 12300, 'position': 'SP', 'efp': -20.388, 'name': 'Carlos Carrasco', 'efp/$1000000': -1657}]
[{'salary': 12000, 'position': 'SP', 'efp': -24.941, 'name': 'Stephen Strasburg', 'efp/$1000000': -2078}]
-24.941


<h2> Starting the Algorithm </h2>
<p> Now that we have an inital global lower bound we can began the algorithm. </p>
<p> We need to now write the main code that will loop through every path starting by selecting a greedy player as the first choice and continuing down that path until a lower and upper bound for that starting pick has been made </p>
<p> For example, given three players(value, weight): Stephen Strasburg(5, 3), Clayton Kershaw(4, 2), and Jose Fernandez(2, 2). <br> <br>
Let's say the weight cap is 4 and there is a max of two people allowed in a set. <br>
After applying the greedy algorithm we would choose Stephen and Clayton which gives us a global lower bound of 8.
If any future set of players has a upper bound less than 8, we will discard that set. <br><br>
Initially we may choose Stephen who has a value of 5 and a weight of 3.
The lower bound is then 5 since the set of players has a value of at least 5 since no player can give negative value His upper bound is 9 (him and Clayton), which we get by sorting the remaining players by value (descending) and taking players until the number of players limit has been met. Thus the upper bound is not less than the global lower bound and we need to continue pursuing more sets that start with Stephen. <br> <br>
Next we would look at Clayton. Adding Clayton would push the weight of players above the threshold of 4 (5). Thus, we cannot use a player set with Stephen and Clayton. The lower bound and upper bound remain the same.<br> <br>
The next player we look at is Jose. We face the same situation and thus cannot use a player set with Stephen and Jose either. There are no more players to combo with so we are done looking at combinations with Stephen in it. His one combination with only him in it remains in the partial sets with a lb (lower bound) of 5 and an ub (upper bound) of 5. The upper bound is now the same as the lower bound since just Stephen is now considered a complete set.<br> <br>
Now we try starting with Clayton. His initial lb is 4 (just him) and his ub his 6 (him and Jose). His upperbound is not less than the global lower bound so we continue with Clayton. We now try to include Jose (the next player) which works since their combined weights do not exceed the limit. The lb of this new partial set is 6 and upper bound is 6 as well since they have two players. The lb bound of the set is now greater than the global lower bound so the global bound is updated to 6 from 5. Note that right now there are three partial sets stored in a max heap: [{Stephen},
{Clayton}, {Clayton, Jose}]. The new global lower bound is then compared with the heap. Since the global lower bound is greater than  or equal to two of the solution's upper bounds, the {Stephen} and {Clayton} solutions are removed and no longer considered. There is no more we can do with Clayton as the intial player. <br> <br>
We now look at Jose. His upper bound is 2 which is lower than the global lower bound so no more partial solutions are added. <br> <br>
There are no more players to start with so the algorithm is done with one solution still being considered. Thus, the set of {Clayton, Jose} is the optimal set of players.
</p>


In [163]:
'''
The inital set S is df_data since it contains all the players/choices.
Now we start by choosing the first players in df_data

Note: The algorithm may look a little different than the example shown above because expected fantasy points is
now a negative field.
'''
import heapq as h
#heap of all partial sets being considered
partial_sets = []
#current set
current_set = Player_Set([])
print df_data
#loop through each possible starting player
#nextPartialSets(0, df_data, partial_sets, current_set)

[{'salary': 12000, 'lb': -24, 'name': 'Stephen Strasburg', 'efp': -24.941, 'efp/$1000000': -2078, 'position': 'SP', 'ub': 0}, {'salary': 14400, 'lb': -28, 'name': 'Clayton Kershaw', 'efp': -28.847, 'efp/$1000000': -2003, 'position': 'SP', 'ub': 0}, {'salary': 13000, 'lb': -26, 'name': 'Jose Fernandez', 'efp': -25.872, 'efp/$1000000': -1990, 'position': 'SP', 'ub': 0}, {'salary': 12300, 'lb': -24, 'name': 'Johnny Cueto', 'efp': -24.35, 'efp/$1000000': -1979, 'position': 'SP', 'ub': 0}, {'salary': 11300, 'lb': -21, 'name': 'Corey Kluber', 'efp': -21.856, 'efp/$1000000': -1934, 'position': 'SP', 'ub': 0}, {'salary': 13700, 'lb': -26, 'name': 'Max Scherzer', 'efp': -26.145, 'efp/$1000000': -1908, 'position': 'SP', 'ub': 0}, {'salary': 12200, 'lb': -22, 'name': 'Rich Hill', 'efp': -22.646, 'efp/$1000000': -1856, 'position': 'SP', 'ub': 0}, {'salary': 12300, 'lb': -20, 'name': 'Carlos Carrasco', 'efp': -20.388, 'efp/$1000000': -1657, 'position': 'SP', 'ub': 0}]


In [201]:
'''
Function to generate all the possible partial sets that can be made by adding one player
'''
def nextPartialSets(player_index, df_data, partial_sets, current_set):
    for player_index in range (0, len(df_data)):
        next_player = df_data[player_index]
        #if there is a possible solution starting with init_player that is better/= to current best add it to partial sol.
        h.heappush(partial_sets, current_set_player)
        #current partial set
        current_set = []
        current_set = [partial_sets[len(partial_sets)-1]]
        #set the lower bound for the partial set with one player, his efs
        current_set["lb"] = int(starting_player["efp"])
        #make the global bound attribute for the new partial set
        current_set["ub"] = 0
        if (getUpperBound(1, current_set, init_player_index, df_data) <= global_lower_bound):
            #loop to continuously add partial sets
            for i in range (init_player_index+1, len(df_data)):
                next_player = df_data[i]
                h.heappush(partial_sets, current_set.append(next_player))

In [202]:
import heapq as h
heap = []
h.heappush(heap, 5)
h.heappush(heap, 2)
h.heappush(heap, 3)
h.heappush(heap, 3)
print heap

[2, 3, 3, 5]


In [203]:
class Player_Set:
    lower_bound = 1
    upper_bound = 1
    
    def __init__(self, list_players):
        self.list_players = list_players
        self.lower_bound = calcLowerBound(list_players)
        
    def setUpperBound(self, upper_bound):
        self.upper_bound = upper_bound
        
    def setLowerBound(self, lower_bound):
        self.lower_bound = lower_bound
        
    def getUpperBound(self):
        return selfupper_bound
    
    def getLowerBound(self):
        return self.lower_bound
    
    def getPlayers(self, player):
        print self.list_players
        print self.list_players.append(player)
    
    def addPlayer(self, player):
        print player
        #if (self.list)
        self.list_players = self.list_players.append(player)
        print self.list_players
        self.lower_bound = calcLowerBound(self.list_players)

In [204]:
'''
Assisiting Functions
''' 
def getUpperBound(set_size, partial_set, init_player_index, df_data):
    upperBound = partial_set["lb"]
    remainingSet = []
    #check that there is still players left to add the player_limit hasn't been exceeded
    if (init_player_index < len(df_data) and set_size <= player_limit):
        #sort remaining players that could be added to the player_set by efp
        remainingSet = sortLODBy("efp", df_data[init_player_index+1:len(df_data)])
        #keep track of remaining player index
        remaining_player_index = 0
        while (set_size < player_limit and remaining_player_index < len(remainingSet)):
            upperBound += remainingSet[remaining_player_index]["efp"]
            set_size += 1
            remaining_player_index +=1
    else:
        return 0
    return upperBound

def calcLowerBound(list_players):
        lower_bound = 0
        if (len(list_players) > 0):
            for player in list_players:
                lower_bound += player["efp"]
        return lower_bound

In [205]:
PAS = Player_Set(df_data)
print df_data
del PA
#PA.getPlayers(df_data[0])


[{'salary': 12000, 'lb': -24, 'name': 'Stephen Strasburg', 'efp': -24.941, 'efp/$1000000': -2078, 'position': 'SP', 'ub': 0}, {'salary': 14400, 'lb': -28, 'name': 'Clayton Kershaw', 'efp': -28.847, 'efp/$1000000': -2003, 'position': 'SP', 'ub': 0}, {'salary': 13000, 'lb': -26, 'name': 'Jose Fernandez', 'efp': -25.872, 'efp/$1000000': -1990, 'position': 'SP', 'ub': 0}, {'salary': 12300, 'lb': -24, 'name': 'Johnny Cueto', 'efp': -24.35, 'efp/$1000000': -1979, 'position': 'SP', 'ub': 0}, {'salary': 11300, 'lb': -21, 'name': 'Corey Kluber', 'efp': -21.856, 'efp/$1000000': -1934, 'position': 'SP', 'ub': 0}, {'salary': 13700, 'lb': -26, 'name': 'Max Scherzer', 'efp': -26.145, 'efp/$1000000': -1908, 'position': 'SP', 'ub': 0}, {'salary': 12200, 'lb': -22, 'name': 'Rich Hill', 'efp': -22.646, 'efp/$1000000': -1856, 'position': 'SP', 'ub': 0}, {'salary': 12300, 'lb': -20, 'name': 'Carlos Carrasco', 'efp': -20.388, 'efp/$1000000': -1657, 'position': 'SP', 'ub': 0}, {'salary': 12000, 'lb': -24, '

NameError: name 'PA' is not defined

In [197]:
print PA.getLowerBound()

NameError: name 'PA' is not defined